In [1]:
import os
import cv2
import numpy as np
from PIL import Image

def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Adaptive thresholding on grayscale image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)

    return thresh

def extract_alphabets_from_images(image_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    successful_separations = set()

    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            continue
        
        # Save a copy of the original image
        original_image_path = os.path.join(output_folder, f"{image_file}")
        cv2.imwrite(original_image_path, image)
        
        # Process the image
        processed_image = preprocess_image(image)
        
        # Find contours
        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if 10 < h < 21 and w > 25:
                alphabet_region = image[y:y+h, x:x+w]
                
                # Create a filename based on coordinates
                base_filename = f"{os.path.splitext(image_file)[0]}_x{x}_y{y}_w{w}_h{h}.png"
                filename = os.path.join(output_folder, base_filename)
                
                # Save the segmented alphabet image
                alphabet_image = Image.fromarray(alphabet_region)
                alphabet_image.save(filename)
                
                # Crop the original image along the y-coordinate
                upper_part = image[:y, :]
                lower_part = image[y:, :]
                
                upper_part_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_upper.png")
                lower_part_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_lower.png")
                
                # Save the cropped images
                cv2.imwrite(upper_part_path, upper_part)
                cv2.imwrite(lower_part_path, lower_part)

                # Replace the original image with the upper part
                os.remove(image_path)
                cv2.imwrite(image_path, upper_part)
                
                # Add the image name to the set of successful separations
                successful_separations.add(image_file)
                
                break  # Assuming only one contour meets the criteria per image
    
    print(f"Segmentation completed. Check the output folder: {output_folder}")
    print(f"Images with successful separations: {successful_separations}")

# Example usage
image_folder = "mixed"  # Folder containing images
output_folder = "inte_mixed"  # Output folder
extract_alphabets_from_images(image_folder, output_folder)

Segmentation completed. Check the output folder: inte_mixed
Images with successful separations: {'alphabet_3_103_582.png', '2.png', 'alphabet_14_209_420.png', 'alphabet_4_448_627.png', 'alphabet_17_481_670.png', 'alphabet_28_178_647.png', 'alphabet_21_485_199.png', 'alphabet_27_130_677.png', 'alphabet_160_102_501.png', 'alphabet_21_101_404.png', 'alphabet_21_486_496.png', 'alphabet_105_108_284.png', 'alphabet_5_311_335.png', 'alphabet_21_487_112.png', 'alphabet_20_489_610.png', 'alphabet_28_284_556.png', 'alphabet_28_215_551.png', 'alphabet_28_249_557.png', 'alphabet_28_138_644.png', 'alphabet_62_325_661.png', 'alphabet_20_228_131.png', 'alphabet_28_138_553.png', 'alphabet_21_279_673.png', 'alphabet_88_117_173.png', 'alphabet_2_433_499.png', 'alphabet_81_221_80.png', 'alphabet_130_389_90.png', 'alphabet_32_425_562.png', '97.png', 'alphabet_66_101_489.png', 'alphabet_174_157_573.png', 'alphabet_25_117_678.png', 'alphabet_27_134_262.png'}
